# Examining Concentrated Liquidity Techniques

## Goals
- Understand the value of re-balancing
    - Explore both "chasing" and Geometric Mean rebalancing
- Look at tight ranges and re-balancing vs larger ranges
- Need to understand both bull and bear cases

### Each frame relies on data from a previous one, so make sure all frames have been run

### First step is to get historical data to use for price predictions

In [ ]:
# Module imports
import helper_classes as hc
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Keep the historical fetching below 90 days to get hourly based data
api_id = 'bitcoin'
end = pd.Timestamp(dt.today())
start = end - pd.DateOffset(days=80)  # Get previous years worth of data
btc_data = pd.DataFrame(hc.get_historical_prices(api_id, start, end)).T

api_id = 'ethereum'
end = pd.Timestamp(dt.today())
start = end - pd.DateOffset(days=80)  # Get previous years worth of data
eth_data = pd.DataFrame(hc.get_historical_prices(api_id, start, end)).T

api_id = 'solana'
end = pd.Timestamp(dt.today())
start = end - pd.DateOffset(days=80)  # Get previous years worth of data
sol_data = pd.DataFrame(hc.get_historical_prices(api_id, start, end)).T

### Create a prediction model using the Weiner process
- Start with Bitcoin
- Can use the historical price chart to decide how much data you want to include to determine the mean and variance used in the model
    - 30 days is what we'll start with

In [ ]:
# Use Weiner process to get future price predictions
lookback_days = 7  # Decide how much historical data to use for prediciton input
lookback_hours = lookback_days * 24
data = btc_data.copy().tail(lookback_hours)
data['gain'] = data['price'].pct_change()
mean_gain = data['gain'].mean()
var_multiplier = 1.2  # use this to get a wider range of outcomes
std_gain = data['gain'].std() * var_multiplier

btc = hc.Token("BTC", 101000)
iterations = 2500  # Can adjust this for more variance if desired
predict_days = 14
predict_hours = predict_days * 24
b = hc.Brownian()
predict = []
for i in range(iterations):
    x = b.stock_price(btc.price, mean_gain, std_gain, predict_hours, 1)
    predict.append(x)
df = pd.DataFrame(predict).T

# Look at price prediction over time
mean_price = df.mean(axis=1)
min_price = df.min(axis=1)
max_price = df.max(axis=1)
std_price = df.std(axis=1)

plt.plot(df.index, mean_price, label='Mean')
plt.plot(df.index, min_price, label='Min')
plt.plot(df.index, max_price, label='Max')
plt.title('Bitcoin Price Prediction - Bounds')
plt.xlabel('Time (hours)')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

In [ ]:
df.plot(legend=False, ylabel='Price(USD)', xlabel='Hours',title='Bitcoin Price Prediction Simulations')

### Next we need to set up our starting conditions for our investment and understand the potential fees earned
- Fees need to be taken from the LP in use, such as Aerodrome, Uniswap, or PancakeSwap

In [ ]:
# cbBTC/USDC Farm stats on Aerodrome
tick_spacing = 100
weekly_rewards = 400652
tvl_rewarded = 1.8E6  # This can change quite a bit and determines how "concentrated" the pool is
apr_per_tick = weekly_rewards / tvl_rewarded * 52 * 100
seed = 100000
fee_per_hour_per_tick = apr_per_tick / 100 / 365 / 24 * seed
print(f"Fee per hour, per liquidity tick: ${fee_per_hour_per_tick:.2f}")

# Decide on ranges to use.  
high_pct = 2
low_pct = 2
# Convert to ticks based on spacing
high_tick = int(high_pct * tick_spacing / 100)
low_tick = int(low_pct * tick_spacing / 100)
fee_per_hour = fee_per_hour_per_tick / (high_tick + low_tick + 1)
print(f"Fee per hour: ${fee_per_hour:.2f}")

il = []
val = []
rebalance = True
for label, sim in df.items():  # Each column is a simulation
    # Set up LP
    btc = hc.Token("BTC", 106500)
    usdc = hc.Token("USDC", 1)
    btc_usdc = hc.LiquidityPool(btc, usdc)
    btc_usdc.gm_rebalance = False
    btc_usdc.setup_new_position(seed, low_tick, high_tick)
    for price in sim.iloc[1:].values:  # start after the first hour
        btc.price = price
        btc_usdc.update_token_balances(1/24)
        if btc_usdc.in_range:
            btc_usdc.fees_accrued += fee_per_hour * 0.982  # VFAT charges 1.8% fee on AERO rewards
        elif rebalance:
            btc_usdc.rebalance(low_tick, high_tick)
        else:
            pass
    il.append(btc_usdc.impermanent_loss)
    val.append(seed - btc_usdc.value)
loss = pd.Series(il)
loss.hist(bins=50)
vs_usd_hold = pd.Series(val)
vs_usd_hold.hist(bins=50)

In [ ]:
# Ethereum
# Use Weiner process to get future price predictions
lookback_days = 75  # Decide how much historical data to use for prediciton input
lookback_hours = lookback_days * 24
data = eth_data.copy().tail(lookback_hours)
data['gain'] = data['price'].pct_change()
mean_gain = data['gain'].mean()
var_multiplier = 1.2  # use this to get a wider range of outcomes
std_gain = data['gain'].std() * var_multiplier

start_price = 3300
iterations = 2500  # Can adjust this for more variance if desired
predict_days = 14
predict_hours = predict_days * 24
b = hc.Brownian()
predict = []
for i in range(iterations):
    x = b.stock_price(start_price, mean_gain, std_gain, predict_hours, 1)
    predict.append(x)
df = pd.DataFrame(predict).T

# Look at price prediction over time
mean_price = df.mean(axis=1)
min_price = df.min(axis=1)
max_price = df.max(axis=1)
std_price = df.std(axis=1)

plt.plot(df.index, mean_price, label='Mean')
plt.plot(df.index, min_price, label='Min')
plt.plot(df.index, max_price, label='Max')
plt.title('Ethereum Price Prediction - Bounds')
plt.xlabel('Time (hours)')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

In [ ]:
# WETH/USDC Farm stats on Aerodrome
tick_spacing = 100
weekly_rewards = 1308872
tvl_rewarded = 6E6  # This can change quite a bit and determines how "concentrated" the pool is
apr_per_tick = weekly_rewards / tvl_rewarded * 52 * 100
seed = 100000
fee_per_hour_per_tick = apr_per_tick / 100 / 365 / 24 * seed
print(f"Fee per hour, per liquidity tick: ${fee_per_hour_per_tick:.2f}")

# Decide on ranges to use.  
high_pct = 3
low_pct = 3
# Convert to ticks based on spacing
high_tick = int(high_pct * tick_spacing / 100)
low_tick = int(low_pct * tick_spacing / 100)
fee_per_hour = fee_per_hour_per_tick / (high_tick + low_tick + 1)
print(f"Fee per hour: ${fee_per_hour:.2f}")

il = []
val = []
rebalance = True
for label, sim in df.items():  # Each column is a simulation
    # Set up LP
    eth = hc.Token("ETH", 3300)
    usdc = hc.Token("USDC", 1)
    eth_usdc = hc.LiquidityPool(eth, usdc)
    eth_usdc.gm_rebalance = True
    eth_usdc.setup_new_position(seed, low_tick, high_tick)
    for price in sim.iloc[1:].values:  # start after the first hour
        eth.price = price
        eth_usdc.update_token_balances(1/24)
        if eth_usdc.in_range:
            eth_usdc.fees_accrued += fee_per_hour * 0.982  # VFAT charges 1.8% fee on AERO rewards
        elif rebalance:
            eth_usdc.rebalance(low_tick, high_tick)
        else:
            pass
    il.append(eth_usdc.impermanent_loss)
    val.append(seed - eth_usdc.value)
loss = pd.Series(il)
loss.hist(bins=50)
final_value = pd.Series(val)
final_value.hist(bins=50)

In [ ]:
# Try to do a manual 30% reduction


btc = hc.Token("BTC", 101000)
iterations = 2500  # Can adjust this for more variance if desired
predict_days = 60
predict_hours = predict_days * 24
b = hc.Brownian()
predict = {}
gain = [-0.3 / 60 / 24, 0.43 / 60 / 24]
start_price = [100000, 70000] 
std_gain = 0.01
for j in range(2):
    predict[j] = []
    btc.price = start_price[j]
    mean_gain = gain[j]
    offset = j * predict_hours
    for i in range(iterations):
        x = b.stock_price(btc.price, mean_gain, std_gain, predict_hours, 1)
        predict[j].append(x)
df1  = pd.DataFrame(predict[0]).T
df2  = pd.DataFrame(predict[1]).T
df = pd.concat([df1,df2], ignore_index=True)

# Look at price prediction over time
mean_price = df.mean(axis=1)
min_price = df.min(axis=1)
max_price = df.max(axis=1)
std_price = df.std(axis=1)

df.plot(legend=False, ylabel='Price(USD)', xlabel='Hours',title='Bitcoin Price Prediction Simulations')

In [ ]:
# cbBTC/USDC Farm stats on Aerodrome
tick_spacing = 100
weekly_rewards = 393721
tvl_rewarded = 2.1E6  # This can change quite a bit and determines how "concentrated" the pool is
apr_per_tick = weekly_rewards / tvl_rewarded * 52 * 100
seed = 100000
fee_per_hour_per_tick = apr_per_tick / 100 / 365 / 24 * seed
print(f"Fee per hour, per liquidity tick: ${fee_per_hour_per_tick:.2f}")

# Decide on ranges to use.  
high_pct = 2
low_pct = 2
# Convert to ticks based on spacing
high_tick = int(high_pct * tick_spacing / 100)
low_tick = int(low_pct * tick_spacing / 100)
fee_per_hour = fee_per_hour_per_tick / (high_tick + low_tick + 1)
print(f"Fee per hour: ${fee_per_hour:.2f}")

il = []
usd_hold = []
best_price = []
final_val = []
rebalance = True
for label, sim in df.items():  # Each column is a simulation
    # Set up LP
    btc = hc.Token("BTC", 106500)
    usdc = hc.Token("USDC", 1)
    btc_usdc = hc.LiquidityPool(btc, usdc)
    btc_usdc.gm_rebalance = False
    btc_usdc.setup_new_position(seed, low_tick, high_tick)
    min = sim.min()
    for price in sim.iloc[1:].values:  # start after the first hour
        btc.price = price
        btc_usdc.update_token_balances(1/24)
        if btc_usdc.in_range:
            btc_usdc.fees_accrued += fee_per_hour * 0.982  # VFAT charges 1.8% fee on AERO rewards
        elif rebalance:
            btc_usdc.rebalance(low_tick, high_tick)
        else:
            pass
    il.append(btc_usdc.impermanent_loss)
    usd_hold.append(seed - btc_usdc.value)
    final_val.append(btc_usdc.value)
    best_price.append(seed / min * btc.price)  # buy at the low point, sell at the final point
loss = pd.Series(il)
loss.hist(bins=50)
vs_usd_hold = pd.Series(usd_hold)
vs_usd_hold.hist(bins=50)

In [ ]:
final_value = pd.Series(final_val)
final_value.hist(bins=50)

In [ ]:
market_time = pd.Series(best_price)
market_time.hist(bins=50)

In [ ]:
(market_time - final_value).hist(bins=50)

In [2]:
import core.helper_classes as hc

seed = 10000
low_tick = 2
high_tick = 2

btc = hc.Token("BTC", 97000)
usdc = hc.Token("USDC", 1)
btc_usdc = hc.LiquidityPool(btc, usdc)
btc_usdc.gm_rebalance = False
btc_usdc.setup_new_position(seed, low_tick, high_tick)
print(btc_usdc)
btc.price = 97000 * 0.87
btc_usdc.update_token_balances(1)
btc_usdc.rebalance(low_tick, high_tick)
print(btc_usdc)
btc.price = 97000
btc_usdc.update_token_balances(1)
btc_usdc.rebalance(low_tick, high_tick)
print(btc_usdc)
btc.price = 97000 * 1.13
btc_usdc.update_token_balances(1)
btc_usdc.rebalance(low_tick, high_tick)
print(btc_usdc)
btc.price = 97000
btc_usdc.update_token_balances(1)
btc_usdc.rebalance(low_tick, high_tick)
print(btc_usdc)


BTC: 0.043700 ($4238.91)| USDC: 5750.000000 ($5750.00)
LP Value: $9988.91
Current Price: 97000.000000 in USDC/BTC
Ticks (low, current, high) 114600, 114830, 115000
Range: 94790.743227 ~ 98659.029540
Hold Value: $9988.91
Impermanent Loss: $0.00
Fee APR required to offset IL: 0.0%
Fees Accrued: $0.00
Total Fees: $0.00
Dust returned: $0.00

BTC: 0.041751 ($3523.37)| USDC: 5204.722909 ($5204.72)
LP Value: $8728.09
Current Price: 84390.000000 in USDC/BTC
Ticks (low, current, high) 113199, 113437, 113599
Range: 82399.450133 ~ 85762.063973
Hold Value: $8673.52
Impermanent Loss: $-54.57
Fee APR required to offset IL: 0.0%
Fees Accrued: $0.00
Total Fees: $0.00
Dust returned: $19.34

BTC: 0.038039 ($3689.81)| USDC: 5056.442416 ($5056.44)
LP Value: $8746.25
Current Price: 97000.000000 in USDC/BTC
Ticks (low, current, high) 114599, 114830, 114999
Range: 94781.265101 ~ 98649.164623
Hold Value: $9960.07
Impermanent Loss: $1213.82
Fee APR required to offset IL: 2552.6%
Fees Accrued: $0.00
Total Fees: